In [1]:
from model import *
from preprocessing import *
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim
from tqdm import tqdm
import lightning as L




In [5]:
model=AlbertModel()
torch.cuda.empty_cache()

In [14]:


# define the LightningModule
class LitAutoEncoder(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model


    def training_step(self, batch):
        # training_step defines the train loop.
        # it is independent of forward
        
        # batch={k:v.to(device) for k,v in batch.items()}
        # send batch to device
        
        input_ids,attention_mask,token_type_ids=batch['input_ids'],batch['attention_mask'],batch['token_type_ids']
        # y_mlm=batch['labels'].float().requires_grad_(True)
        y_mlm=batch['input_ids']
        y_nsp=batch['label'].squeeze().float().requires_grad_(True)
        
        y_hat_mlm,y_hat_nsp = self.model((input_ids,attention_mask,token_type_ids))
        y_hat_mlm=y_hat_mlm.transpose(1,2)
        y_hat_nsp=y_hat_nsp
        # print(y_hat_nsp,y_nsp)
        # print(y_hat_mlm.shape,y_mlm.shape)
        print(y_hat_nsp.shape,y_nsp.shape)
        # print(y_hat_mlm.dtype,y_mlm.dtype)
        # print(y_hat_nsp.dtype,y_nsp.dtype)
        loss_mlm = nn.functional.nll_loss( y_hat_mlm,y_mlm.squeeze(-1))
        loss_nsp = nn.functional.nll_loss( y_hat_nsp,y_nsp)
        # print(loss_mlm,loss_nsp)
        loss=torch.mean(loss_mlm+loss_nsp)
        print(loss)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss,prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=5e-3)
        return optimizer
    def predict_step(self, batch ):
        input_ids,attention_mask,token_type_ids=batch['input_ids'],batch['attention_mask'],batch['token_type_ids']
        y=self.model((input_ids,attention_mask,token_type_ids))
        
        # y_mlm=batch['input_ids']
        # y_nsp=batch['label'].squeeze()
        
        y_hat_mlm,y_hat_nsp = self((input_ids,attention_mask,token_type_ids))
        y_hat_mlm=y_hat_mlm.argmax(-1)
        y_hat_nsp=y_hat_nsp.argmax(-1)
        return y_hat_mlm,y_hat_nsp

# init the autoencoder
bert = LitAutoEncoder(model)
trainer = L.Trainer(limit_train_batches=100, max_epochs=1)
# trainer.fit(model=bert, train_dataloaders=text_dataloader);
bert.predict_step(next(iter(text_dataloader)))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# print(outputs.last_hidden_state.shape)  
# print(outputs.pooler_output.shape)  
print(tokenizer.convert_ids_to_tokens(torch.argmax(outputs.logits,-1).flatten())[1:7])

['▁hello', ',', '▁my', '▁dog', '▁is', '!!']


In [ ]:
print(model)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis